# Spam Classification

The objective of it is to generate a model capable to classify a email as spam or not spam.

The dataset used was from http://www2.aueb.gr/users/ion/data/enron-spam/

## Imports

In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import string
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob
from email.message import EmailMessage
from email.parser import BytesParser, Parser
from email.policy import default

[nltk_data] Downloading package stopwords to /home/phrc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/phrc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Functions

In [47]:
def createDf(path, isSpam):
    """
    Insert all the email context into a dataframe column and classify if the email is a spam or not

    Parameters
    ----------
    path: str
        Path o directory to be read
    isSpam: Boolean
        true is the files are spam or false otherwise        

    Returns
    ----------
    dataframe
        a dataframe with 2 columns isSpam and emailContext
    """
    os.chdir(path)
    data = []
    for root, dirs, files in os.walk(path):
         for file in files:
            with open(os.path.join(root, file), "r") as openFile:
                dictTemp = {}
                try:
                    message = Parser(policy=default).parsestr(openFile.read())
                    dictTemp['file'] = openFile.name
                    for key in message.keys():
                        dictTemp[key.lower()] = message[key]
                    dictTemp['messageType'] = message.get_content_type()
                    if message.get_content_type() == 'text/plain':
                        try:
                            dictTemp['message'] = str(message.get_content())
                        except:
                            dictTemp['message'] = str(message.get_body())
                            dictTemp['messageType'] = 'Error'
                    else:
                        dictTemp['message'] = str(message.get_body())
                    dictTemp['parseError'] = False                    
                except:
                    dictTemp['parseError'] = True
                data.append(dictTemp)
    df = pd.DataFrame(data)
    df['isSpam'] = isSpam
    
#print(headers.keys())

#print(headers.values())
#str(headers.get_content_type())
#str(headers.get_body())
#str(headers.get_content())

    
#df.columns = ['text', 'isSpam', 't1', 't2']
    return df

def emailTextCleanner(text):
    """
    Remove the first apperance of the word 'Subject:'

    Parameters
    ----------
    text: str

    Returns
    ----------
    str
    """
    text = text.replace('Subject:', '', 1)
    #print(text)
    #print("---------------------------------------------")
    #print("---------------------------------------------")
    #text = TextBlob(text)
    #text = str(text.correct())
    #print(text)
    #print("\n")
    return text

def lemmatizeList(words):
    """
    Lemmatize a list of string

    Parameters
    ----------
    words: list of str

    Returns
    ----------
    list of str
    """
#    print("|", end='')
    lem = WordNetLemmatizer()
    for i in range(len(words)):
#        text = TextBlob(words[i])
#        words[i] = str(text.correct())
        words[i] = lem.lemmatize(words[i], 'v')
        words[i] = lem.lemmatize(words[i], 'n')
    return words

def revomeWordsWithOneCharacter(words):
    return list(filter(lambda x : len(x) > 1, words))

def removeDigits(words):
    return list(filter(lambda x : x.isdigit() == False, words))

In [48]:
#path = '/home/phrc/Python Project/Spam Email Classifier/emails/ham/beck-s/2001_plan/'
#path = '/home/phrc/Python Project/Spam Email Classifier/emails/ham/beck-s/2001_plan/1'
path = '/home/phrc/Python Project/Spam Email Classifier/emails/spam/'
#path = '/home/phrc/Python Project/Spam Email Classifier/emails/spam/BG/2004/08/1091394468.23940_19.txt'

dfEnron2Spam = createDf(path, True)

dfEnron2Spam.head()

#df = dfEnron2Ham[dfEnron2Ham['fileErro'].notnull()]

#df.head()
#f = open(path,"r")

#type(f)

#files = [file for file in os.listdir(path) if os.path.isfile(file)]
#for file in files:
#    print(path+file)
#    break

#print(os.path.dirname(f))

#headers = Parser(policy=default).parsestr(f.read())

#print(headers.keys())

#print(headers.values())
#if headers.get_content_type() == 'text/plain':
#    print('OK')
#str(headers.get_body())
#str(headers.get_content())



#t = df.head(1)['text']
#type(t[0])
#print(t[0])


,0x-accept-language,alternate-recipient,andorra-islamabad,approved,approved-by,archenemy-matthew,auto-forwarded,auto-submitted,basemen-cankerworm,bcc,...,x-wm-posted-at,x-xam3-api-version,x-yoursite-mailscanner,x-yoursite-mailscanner-information,x-zinester-pid,x_id,x_uid,xmailing-id,xref,isSpam
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [49]:


len(dfEnron2Spam)

df = dfEnron2Spam[dfEnron2Spam['content-description'].notnull()]
print(len(df))

#print(df['message'])


27


In [50]:
len(dfEnron2Spam)


10000

In [57]:
df = dfEnron2Spam.describe(include=['object']).T

#df.head(300)
#len(df[df['count']>1000])

print(len(dfEnron2Spam))
#df[df['count']>1000].head(30)


df.loc['parseError'].head()
#len(dfEnron2Spam)



10000


KeyError: 'parseError'

## Reading and preparing the dataset

In [21]:
#Change this line to your computer path from the dataset
path = '/Users/phrc/Documents/Projects/pythonProject/ML assement/enron2/'

#Create ham and spam df  
dfEnron2Ham = createDf(path+'ham/', False)
dfEnron2Spam = createDf(path+'spam/', True)

#Append ham and spam df
df = dfEnron2Ham.append(dfEnron2Spam)

#Remove 'Subject:'  
print('Start spelling \n corrector total files: {} \n'.format(len(df['text'])))
df['treatedText'] = df['text'].apply(emailTextCleanner)

#Remove Stop Words
stop = text.ENGLISH_STOP_WORDS
pat = r'\b(?:{})\b'.format('|'.join(stop))
df['treatedText'] = df['treatedText'].str.replace(pat, '')


#Remove punctuation
df['treatedText'] = df['treatedText'].str.replace('[{}]'.format(string.punctuation), '')

#Remove extra space and break lines
df['treatedText'] = df['treatedText'].str.replace('\n', ' ')

#Create a column with list of words
df['wordsList'] = df['treatedText'].str.split().apply(lemmatizeList).apply(revomeWordsWithOneCharacter).apply(removeDigits)



#Create a columns to calculate the total amount of words
df['totalTreatedWords'] = df['wordsList'].apply(lambda x : len(x))
df['treatedTextLen'] = df['treatedText'].apply(lambda x : len(x))
df['textLen'] = df['text'].apply(lambda x : len(x))
df['uniqueWordsLen'] = df['wordsList'].apply(lambda x : len(set(x)))

df.head()

Start spelling 
 corrector total files: 5857 

 re : telephone interview with enron corp . research dept .
dear shirley :
confirming that i will be waiting for the telephone interview at 1 pm
tomorrow . ? i would like to give you my cell phone number , 713 / 907 - 6717 , as a
back - up measure . ? please note that my first preference is to receive the call
at my home number , 713 / 669 - 0923 .
sincerely ,
rabi de
?
? shirley . crenshaw @ enron . com wrote :
dear rabi :
i have scheduled the telephone interview for 1 : 00 pm on friday , july 7 th .
we will call you at 713 / 669 - 0923 . if there are any changes , please let
me know .
sincerely ,
shirley crenshaw
713 - 853 - 5290
rabi deon 06 / 26 / 2000 10 : 37 : 24 pm
to : shirley crenshaw
cc :
subject : re : telephone interview with enron corp . research dept .
dear ms . crenshaw :
thanks for your prompt response . ? july 6 or 7 th will work best for me . . ? i
would prefer to be called at my home number . ? please let me know the
sch

 re : into help .
krishna ,
nicholas introduces himself as an associate in the research group .
i think we should clarify his status .
since
- - - - - - - - - - - - - - - - - - - - - - forwarded by since j kamenski / you / act on 08 / 15 / 2000
05 : 53 pm - - - - - - - - - - - - - - - - - - - - - - - - - - -
" michael schilmoeller " on 08 / 15 / 2000 11 : 08 : 06
am
to : notes : nicholas . ear @ union
c : vkamins @ union . com , grant _ mason @ pen . com , stanton _ dinner @ pen . com
subject : re : into help .
hi nicholas ,
i am in the middle of preparing some presentations right now , so it might be
more productive to speak by phone ( 503 - 464 - 8430 ) . please leave your number ,
if you get my voicemail .
to get you started , you might see if you can get access to the fera lads
database of plant forced and planned availability . it seems others in
research have asked about this , so you may already have this at your
disposal . the era has a good electronic database of plant for and

KeyboardInterrupt: 

## Visualisation

### Dataset distribution by spam type

In [ ]:
print(df['isSpam'].value_counts().rename({False: 'Ham', True: 'Spam'}))


fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,2, wspace =0.1)
barPlt = fig.add_subplot(grid[0,0])
piePlt = fig.add_subplot(grid[0,1])

barPlt.bar(np.arange(2), df['isSpam'].value_counts(), align='center')
barPlt.set_xticks(np.arange(2), ('Ham', 'Spam'))


piePlt.pie(df['isSpam'].value_counts(), labels=['Ham', 'Spam'], autopct='%1.0f%%', pctdistance=0.5, labeldistance=1.2)


plt.show()

### Compare text lenght for ham and spam  

There is 4 differents attributes what define the text lengh:

- textLen : This is the raw text without any treatment
- treatedTextLen: This is the text after removing the stop words and ponctuation
- totalTreatedWords: This is the amount of words used in the text after the treatment, basically this exclude space and line breaks
- uniqueWordsLen: This is the amount of unique words used in the treated text, basically it removes repited words

### Raw Text size lengh comparison  

In [ ]:
fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,4, wspace =0.8)
histPlt = fig.add_subplot(grid[0,0:2])
boxPlt1 = fig.add_subplot(grid[0,2])
boxPlt2 = fig.add_subplot(grid[0,3])

histPlt.hist(
    [
        df[df['isSpam'] == True]['textLen'], 
        df[df['isSpam'] == False]['textLen']
    ], 
    np.linspace(0, df['textLen'].quantile(0.75), 30), 
    density = True, 
    label=['Spam', 'Ham']
) 
histPlt.legend(loc='upper right')

boxPlt1.boxplot(
    (
         df[df['isSpam'] == True]['textLen'], 
         df[df['isSpam'] == False]['textLen'] 
    ), 
    labels = ('Spam', 'Ham')
)

boxPlt2.boxplot(
    (
        df[(df['isSpam'] == True) & (df['textLen'] < df['textLen'].quantile(0.75))]['textLen'], 
        df[(df['isSpam'] == False) & (df['textLen'] < df['textLen'].quantile(0.75))]['textLen'] 
    ), 
    labels = ('Spam', 'Ham')
)

plt.show()

### Text size without stop words and punctuation 

In [ ]:
fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,4, wspace =0.8)
histPlt = fig.add_subplot(grid[0,0:2])
boxPlt1 = fig.add_subplot(grid[0,2])
boxPlt2 = fig.add_subplot(grid[0,3])

histPlt.hist(
    [
        df[df['isSpam'] == True]['treatedTextLen'], 
        df[df['isSpam'] == False]['treatedTextLen']
    ],  
    np.linspace(0, df['treatedTextLen'].quantile(0.75), 30),
    density= True,
    label=['Spam', 'Ham']
) 
histPlt.legend(loc='upper right')

boxPlt1.boxplot(
    (
        df[df['isSpam'] == True]['treatedTextLen'], 
        df[df['isSpam'] == False]['treatedTextLen'] ), 
    labels = ('Spam', 'Ham')
)

boxPlt2.boxplot(
    (
        df[(df['isSpam'] == True) & (df['treatedTextLen'] < df['treatedTextLen'].quantile(0.75))]['treatedTextLen'], 
        df[(df['isSpam'] == False) & (df['treatedTextLen'] < df['treatedTextLen'].quantile(0.75))]['treatedTextLen'] 
    ), 
    labels = ('Spam', 'Ham')
)

plt.show()



### Total words used

In [ ]:
fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,4, wspace =0.8)
histPlt = fig.add_subplot(grid[0,0:2])
boxPlt1 = fig.add_subplot(grid[0,2])
boxPlt2 = fig.add_subplot(grid[0,3])

histPlt.hist(
    [
        df[df['isSpam'] == True]['totalTreatedWords'], 
        df[df['isSpam'] == False]['totalTreatedWords']
    ],  
    np.linspace(0, df['totalTreatedWords'].quantile(0.75), 30),
    density= True,
    label=['Spam', 'Ham']
) 
histPlt.legend(loc='upper right')

boxPlt1.boxplot(
    (
        df[df['isSpam'] == True]['totalTreatedWords'], 
        df[df['isSpam'] == False]['totalTreatedWords'] 
    ), 
    labels = ('Spam', 'Ham')
)

boxPlt2.boxplot(
    (
        df[(df['isSpam'] == True) & (df['totalTreatedWords'] < df['totalTreatedWords'].quantile(0.75))]['totalTreatedWords'], 
        df[(df['isSpam'] == False) & (df['totalTreatedWords'] < df['totalTreatedWords'].quantile(0.75))]['totalTreatedWords'] 
    ), 
    labels = ('Spam', 'Ham')
)

plt.show()


### Total unique words used

In [ ]:
fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,4, wspace =0.8)
histPlt = fig.add_subplot(grid[0,0:2])
boxPlt1 = fig.add_subplot(grid[0,2])
boxPlt2 = fig.add_subplot(grid[0,3])

histPlt.hist(
    [
        df[df['isSpam'] == True]['uniqueWordsLen'], 
        df[df['isSpam'] == False]['uniqueWordsLen']
    ],  
    np.linspace(0, df['uniqueWordsLen'].quantile(0.75), 30),
    density= True,
    label=['Spam', 'Ham']
) 
histPlt.legend(loc='upper right')

boxPlt1.boxplot(
    (
        df[df['isSpam'] == True]['uniqueWordsLen'], 
        df[df['isSpam'] == False]['uniqueWordsLen'] 
    ), 
    labels = ('Spam', 'Ham')
)

boxPlt2.boxplot(
    (
        df[(df['isSpam'] == True) & (df['uniqueWordsLen'] < df['uniqueWordsLen'].quantile(0.75))]['uniqueWordsLen'], 
        df[(df['isSpam'] == False) & (df['uniqueWordsLen'] < df['uniqueWordsLen'].quantile(0.75))]['uniqueWordsLen'] 
    ), 
    labels = ('Spam', 'Ham')
)

plt.show()


### Conclusion

# Need to do 

----It's not clear the text size can influence into the classification of the email in spam or ham.



## Words Visualization

### Total uniques words

In [ ]:
count = Counter(list(chain(*df["wordsList"].values)))

countSpam = Counter(list(chain(*df[df['isSpam'] == True]["wordsList"].values)))

countHam = Counter(list(chain(*df[df['isSpam'] == False]["wordsList"].values)))

fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,2, wspace =0.2)
pltBar = fig.add_subplot(grid[0,0])
pltText = fig.add_subplot(grid[0,1])


pltBar.bar(
    ['Total unique words', 'Spam unique words', 'Ham unique words'], 
    [len(count), len(countSpam), len(countHam)], 
    align='center'
)

textWords = ['Total unique words:              {}'.format(len(count)), 
             'Total spam unique words:    {}'.format(len(countSpam)), 
             'Total ham unique words:      {}'.format(len(countHam))]


pltText.text(x=0, y=0.5, s = '\n'.join(textWords), fontsize = 18) 
pltText.axis('off')
plt.show()

### 20 Most common words

In [ ]:
dfWords = pd.DataFrame(list(dict(count).items()))
dfWords.columns = ['word', 'occur'] 
totalOcurr = dfWords['occur'].sum()
dfWords['freq'] = dfWords['occur'] / totalOcurr  
dfWords = dfWords.sort_values(by='freq', ascending=False)
dfWords = dfWords.reset_index(drop=True)
dfWords['freqAcum'] = dfWords['freq'].cumsum()


fig = plt.figure(figsize = (18,12))
grid = plt.GridSpec(4,2, wspace =0.2, hspace = 0.5)
pltLine = fig.add_subplot(grid[0:2,0])
pltBar = fig.add_subplot(grid[2:4,0:2])
pltTable = fig.add_subplot(grid[0:2,1])

pltLine.plot(dfWords['freqAcum'], range(len(dfWords)))
pltLine.set_ylabel('Number of Words')
pltLine.set_xlabel('Acumulative Frequence')
pltLine.set_title('Line Graph of acumulative frequence')
pltLine.grid(True)


dfTWord = dfWords.loc[[0,20,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltTable.table(cellText=dfTWord.values, rowLabels= dfTWord.index, colLabels = dfTWord.columns, loc='best')
pltTable.axis('off')
pltTable.set_title('Table of acumulative frequence')



pltBar.bar(dfWords.head(20)['word'], dfWords.head(20)['freq'])
pltBar.set_title('20 Most Used word')

yPos = np.arange(20)
pltBar.set_xticklabels(dfWords.head(20)["word"], rotation=60)
pltBar.set_xticks(yPos)


plt.show()



### 20 Most common words on Spam emails

In [ ]:
dfSpamWords = pd.DataFrame(list(dict(countSpam).items()))
dfSpamWords.columns = ['word', 'occur'] 
totalSpamOcurr = dfSpamWords['occur'].sum()
dfSpamWords['freq'] = dfSpamWords['occur'] / totalSpamOcurr  
dfSpamWords = dfSpamWords.sort_values(by='freq', ascending=False)
dfSpamWords = dfSpamWords.reset_index(drop=True)
dfSpamWords['freqAcum'] = dfSpamWords['freq'].cumsum()


fig = plt.figure(figsize = (18,12))
grid = plt.GridSpec(4,2, wspace =0.2, hspace = 0.5)
pltLine = fig.add_subplot(grid[0:2,0])
pltBar = fig.add_subplot(grid[2:4,0:2])
pltTable = fig.add_subplot(grid[0:2,1])

pltLine.plot(dfSpamWords['freqAcum'], range(len(dfSpamWords)))
pltLine.set_ylabel('Number of Words')
pltLine.set_xlabel('Acumulative Frequence')
pltLine.set_title('Line Graph of acumulative frequence')
pltLine.grid(True)


dfSWord = dfSpamWords.loc[[0,20,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltTable.table(cellText=dfSWord.values, rowLabels= dfSWord.index, colLabels = dfSWord.columns, loc='best')
pltTable.axis('off')
pltTable.set_title('Table of acumulative frequence')



pltBar.bar(dfSpamWords.head(20)['word'], dfSpamWords.head(20)['freq'], align='center')
pltBar.set_title('20 Most Used word')

yPos = np.arange(20)
pltBar.set_xticklabels(dfSpamWords.head(20)["word"], rotation=60)
pltBar.set_xticks(yPos)

plt.show()



### 20 Most common words on Ham emails

In [ ]:
dfHamWords = pd.DataFrame(list(dict(countHam).items()))
dfHamWords.columns = ['word', 'occur'] 
totalHamOcurr = dfHamWords['occur'].sum()
dfHamWords['freq'] = dfHamWords['occur'] / totalHamOcurr  
dfHamWords = dfHamWords.sort_values(by='freq', ascending=False)
dfHamWords = dfHamWords.reset_index(drop=True)
dfHamWords['freqAcum'] = dfHamWords['freq'].cumsum()


fig = plt.figure(figsize = (18,12))
grid = plt.GridSpec(4,2, wspace =0.2, hspace = 0.5)
pltLine = fig.add_subplot(grid[0:2,0])
pltBar = fig.add_subplot(grid[2:4,0:2])
pltTable = fig.add_subplot(grid[0:2,1])

pltLine.plot(dfHamWords['freqAcum'], range(len(dfHamWords)))
pltLine.set_ylabel('Number of Words')
pltLine.set_xlabel('Acumulative Frequence')
pltLine.set_title('Line Graph of acumulative frequence')
pltLine.grid(True)


dfHWord = dfHamWords.loc[[0,20,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltTable.table(cellText=dfHWord.values, rowLabels= dfHWord.index, colLabels = dfHWord.columns, loc='best')
pltTable.axis('off')
pltTable.set_title('Table of acumulative frequence')



pltBar.bar(dfHamWords.head(20)['word'], dfHamWords.head(20)['freq'], align='center')
pltBar.set_title('20 Most Used word')

yPos = np.arange(20)
pltBar.set_xticklabels(dfHamWords.head(20)["word"], rotation=60)
pltBar.set_xticks(yPos)

plt.show()



#### Word List

In [ ]:
tempSet = sorted(set(list(chain(*df[df['isSpam'] == True]["wordsList"].values))))


print(tempSet)

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(tempSet)

print(misspelled)

In [ ]:
lem = WordNetLemmatizer()
words = lem.lemmatize('accommodations')

words

## Modeling

### Spliting prepare data set for modeling 

In [ ]:
l = list(chain(*df["wordsList"].values))

l = set(filter(lambda k : len(k) == 1, l))

l

In [ ]:
dfWordsG = pd.DataFrame(list(dict(count).items()))
dfWordsG.columns = ['word', 'occur'] 
totalOcurrG = dfWordsG['occur'].sum()
dfWordsG['freq'] = dfWordsG['occur'] / totalOcurrG  
dfWordsG = dfWordsG.sort_values(by='freq', ascending=False)
dfWordsG = dfWordsG.reset_index(drop=True)
dfWordsG['freqAcum'] = dfWordsG['freq'].cumsum()

dfSpamWordsG = pd.DataFrame(list(dict(countSpam).items()))
dfSpamWordsG.columns = ['word', 'occur'] 
totalSpamOcurrG = dfSpamWordsG['occur'].sum()
dfSpamWordsG['freq'] = dfSpamWordsG['occur'] / totalOcurrG  
dfSpamWordsG = dfSpamWordsG.sort_values(by='freq', ascending=False)
dfSpamWordsG = dfSpamWordsG.reset_index(drop=True)
dfSpamWordsG['freqAcum'] = dfSpamWordsG['freq'].cumsum()

dfHamWordsG = pd.DataFrame(list(dict(countHam).items()))
dfHamWordsG.columns = ['word', 'occur'] 
totalHamOcurrG = dfHamWordsG['occur'].sum()
dfHamWordsG['freq'] = dfHamWordsG['occur'] / totalOcurrG  
dfHamWordsG = dfHamWordsG.sort_values(by='freq', ascending=False)
dfHamWordsG = dfHamWordsG.reset_index(drop=True)
dfHamWordsG['freqAcum'] = dfHamWordsG['freq'].cumsum()

fig = plt.figure(figsize = (18,6))
grid = plt.GridSpec(1,3, wspace =0.2)
pltTotal = fig.add_subplot(grid[0,0])
pltSpam = fig.add_subplot(grid[0,1])
pltHam = fig.add_subplot(grid[0,2])

pltTotal.plot(dfWords['freqAcum'], range(len(dfWordsG)))
pltTotal.set_ylabel('Total Words')
pltTotal.set_xlabel('Acumulative Frequence')
pltTotal.set_title('All Words')
pltTotal.grid(True)

pltSpam.plot(dfSpamWords['freqAcum'], range(len(dfSpamWordsG)))
pltSpam.set_ylabel('Total Spam Words')
pltSpam.set_xlabel('Acumulative Frequence')
pltSpam.set_title('Spam Words')
pltSpam.grid(True)

pltHam.plot(dfHamWords['freqAcum'], range(len(dfHamWordsG)))
pltHam.set_ylabel('Total Ham Words')
pltHam.set_xlabel('Acumulative Frequence')
pltHam.set_title('Ham Words')
pltHam.grid(True)


fig2 = plt.figure(figsize = (18,6))
grid2 = plt.GridSpec(1,6, wspace =0.4)

pltTotalTableG = fig2.add_subplot(grid2[0,0:2])
pltSpamTableG = fig2.add_subplot(grid2[0,2:4])
pltHamTableG = fig2.add_subplot(grid2[0,4:6])

dfTWordG = dfWordsG.loc[[0,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltTotalTableG.table(cellText=dfTWordG.values, rowLabels= dfTWordG.index, colLabels = dfTWordG.columns, loc='best')
pltTotalTableG.axis('off')

dfSWordG = dfSpamWordsG.loc[[0,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltSpamTableG.table(cellText=dfSWordG.values, rowLabels= dfSWordG.index, colLabels = dfSWordG.columns, loc='best')
pltSpamTableG.axis('off')

dfHWordG = dfHamWordsG.loc[[0,50,100,500,1000,5000,10000,15000]][['freqAcum']]

pltHamTableG.table(cellText=dfHWordG.values, rowLabels= dfHWordG.index, colLabels = dfHWordG.columns, loc='best')
pltHamTableG.axis('off')


plt.show()
